In [9]:
import pandas as pd
import os
import numpy as np

import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, RBF


from sklearn.gaussian_process.kernels import WhiteKernel, RBF
from modAL.models import ActiveLearner, CommitteeRegressor
from modAL.disagreement import max_std_sampling
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
import math

%matplotlib inline

In [14]:
# cleaned version of the data after removing outliers and negative values
path='/workspace/malathi/steam_turbine_stage_1_POC/notebooks/version_1/df_sorted.csv'
data=pd.read_csv(path)
data.shape

(7077, 6)

In [15]:
INPUT_COLUMNS=[ 'expansion ratio','corrected speed','ImpulseStage_CurrentStroke']
OUTPUT_COLUMNS=['ImpulseStage_Power', 'Impulse_Discharge_Temperature', 'corrected mass flow']

In [16]:
data.sample(5)

,expansion ratio,corrected speed,ImpulseStage_CurrentStroke,ImpulseStage_Power,Impulse_Discharge_Temperature,corrected mass flow
6473,1.337366,142.508617,50.65,1641.145019,336.113769,19.231765
7034,1.210939,136.013304,64.97,1286.341431,427.528717,21.071173
5261,1.630553,135.327727,36.35,1851.047241,372.219635,15.706905
4820,1.789481,139.796598,33.48,1986.319824,339.213135,14.301727
3745,2.390737,137.233751,27.76,1575.878662,354.958038,10.492459


In [17]:
# predictions for 'Impulsestage_Power'
X=data.drop(OUTPUT_COLUMNS,axis=1)
y=data['ImpulseStage_Power']
print(X.shape,y.shape)
XX=X.values.reshape(-1,3)
yy=y.values.reshape(-1,1)
print(XX.shape,yy.shape)

(7077, 3) (7077,)
(7077, 3) (7077, 1)


In [23]:
# train_x,test_x,train_y,test_y = train_test_split(XX,yy,test_size=0.85,random_state=999)
# print(train_x.shape,train_y.shape,test_x.shape,test_y.shape)

(1061, 3) (1061, 1) (6016, 3) (6016, 1)


In [30]:
# X_raw=XX.copy()
# y_raw=yy.copy()

# # Isolate our examples for our labeled dataset.
# n_labeled_examples=X_raw.shape[0]
# training_indices = np.random.randint(low=0,high=len(X_raw)+1,size=250)
# print(training_indices[0:5])

# # Defining the training data
# X_training = X_raw[training_indices]
# y_training = y_raw[training_indices]
# print(X_training.shape,y_training.shape)

# # Isolate the non-training examples we'll be querying.
# X_pool = np.delete(X_raw, training_indices, axis=0)
# y_pool = np.delete(y_raw, training_indices, axis=0)
# print(X_pool.shape,y_pool.shape)

[1614 1543 3802 3367 4985]
(250, 3) (250, 1)
(6830, 3) (6830, 1)


In [33]:
n_initial=100
X_train, X_test, y_train, y_test = train_test_split(XX, yy)
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)
#initial_idx = np.random.choice(range(len(X_train)), size=n_initial, replace=False)

#X_initial, y_initial = X_train[initial_idx], y_train[initial_idx]
X_pool, y_pool = np.delete(X_train, initial_idx, axis=0), np.delete(y_train, initial_idx, axis=0)
print(X_pool.shape,y_pool.shape)

(5307, 3) (5307, 1) (1770, 3) (1770, 1)
(5307, 3) (5307, 1)


In [34]:
initial_idx = list()
initial_idx.append(np.random.choice(range(100), size=n_initial, replace=False))
initial_idx.append(np.random.choice(range(100, 200), size=n_initial, replace=False))
learner_list = [ActiveLearner(
                        estimator=GaussianProcessRegressor(kernel),
                        X_training=X_train[idx], y_training=y_train[idx]
                )
                for idx in initial_idx]

/usr/local/lib/python3.8/dist-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


In [35]:
# initializing the Committee
committee = CommitteeRegressor(
    learner_list=learner_list,
    query_strategy=max_std_sampling
)

In [36]:
pred, std = committee.predict(X_pool, return_std=True)
#pred = pred.reshape(-1, )

In [37]:
pred[0:6]

array([1290.17678915, 1463.0164953 , 1146.8332046 , 1465.5236164 ,
       1001.98932075,  601.51227039])

In [38]:
std[0:6]

array([39.94666918, 38.87077788, 17.8939262 , 28.61246271, 23.05294708,
       16.30053903])

In [39]:
# active regression
n_queries = 10
for idx in range(n_queries):
    query_idx, query_instance = committee.query(X_train)
    committee.teach(X_train[query_idx], y_train[query_idx])

/usr/local/lib/python3.8/dist-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The op

In [40]:
pred_1, std_1 = committee.predict(X_pool, return_std=True)

In [41]:
pred_1[0:6]

array([1353.74017672, 1525.96615174, 1154.03909595, 1503.24522575,
       1031.53769449,  647.10609634])

In [42]:
std_1[0:6]

array([31.8174922 , 32.0436094 , 16.84175305, 25.00977792, 17.93389773,
        5.63649528])